# WEB+DB PRESS vol.118 Pythonデータ可視化入門　第3章～第5章

## 参考記事  

- Pythonデータ可視化入門
- 著者：小川英幸さん@ogawahideyuki
- ソースコード：https://github.com/mazarimono/webdb_notebook/

## 対応章

- 第３章　COVID-19/家計調査/財政データの整形
- 第４章　整形したデータの可視化
- 第５章　ブラウザで動くダッシュボードの作成

## ショートカット参考リンク  

### 移動関連

- 矢印キー：移動(セルの最初/最終行を抜ける場合は次のセルに移動)
- Ctrl + M, N：次のセルに移動
- Ctrl + M, P：前のセルに移動

### 編集関連

- Ctrl + M, Y：テキストセルに変換
- Ctrl + M, M：コードセルに変換
- Ctrl + M, A：自セル上部にセル追加
- Ctrl + M, B：自セル下部にセル追加
- Ctrl + M, D：自セルを削除
- Ctrl + M, K：自セル上部セルに移動
- Ctrl + M, J：自セル下部セルに移動

### 実行関連

- Ctrl + Enter：自セル実行
- Ctrl + F9：セル全実行
- Ctrl + Shift + P:コマンドパレット

### その他

- Ctrl + M, H：ショートカット一覧表示
- ツール → 設定 → エディタ → エディタのキーバインディングでVimを設定可能

### 有用リンク

- [Qiita Google Colaboratory コマンド チートシート【jupyter対応】](https://qiita.com/Intel0tw5727/items/7cc68734edd568357a9e)  


In [ ]:
# 事前設定(plotlyのバージョンアップ)
# バージョンアップしないとpath属性が存在しないと怒られるため
!pip show plotly #4.4.1は古い
!pip install -U plotly

Name: plotly
Version: 4.4.1
Summary: An open-source, interactive graphing library for Python
Home-page: https://plot.ly/python/
Author: Chris P
Author-email: chris@plot.ly
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: retrying, six
Required-by: cufflinks
     |████████████████████████████████| 13.1MB 297kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


## 家計調査データの前処理

### Pandasで内容を確認し、データ加工の方針を決める
- 事前作業：総務省統計局の家計調査の時系列データを[ダウンロード](https://github.com/mazarimono/webdb_notebook/tree/master/data/h-mon-a.csv)して配置
- read_csv参考リンク：[詳説Pandasのread_csvとread_table関数の使い方](https://deepage.net/features/pandas-readcsv-deep.html#%E3%83%A1%E3%83%A2%E3%83%AA%E3%83%BC%E3%81%AE%E4%BD%BF%E7%94%A8%E9%87%8F%E3%82%92%E6%8A%91%E3%81%88%E3%82%8B-low_memory)

In [ ]:
import pandas as pd
from datetime import date
#df = pd.read_csv('h-mon-a.csv', encoding='shift-jis') WEB+DB PRESS記載内容
df = pd.read_csv('https://raw.githubusercontent.com/mazarimono/webdb_notebook/master/data/h-mon-a.csv', encoding='shift-jis')
print(df.shape) #次元数確認
print(df.info()) #データの内容確認
print(df.head())
print(df.tail())

(188, 253)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Columns: 253 entries, 二人以上の世帯_支出金額[円] to 247
dtypes: object(253)
memory usage: 371.7+ KB
None
  二人以上の世帯_支出金額[円] Unnamed: 1 Unnamed: 2 Unnamed: 3  ...    244    245    246    247
0             NaN        NaN        NaN        NaN  ...    NaN    NaN    NaN    NaN
1             NaN        NaN        NaN        NaN  ...    NaN    NaN    NaN    NaN
2            表側連番         階層        大分類        中分類  ...     4月     5月     6月     7月
3               1          -          -          -  ...  10000  10000  10000  10000
4               2          -          -          -  ...   7534   7430   7489   7521

[5 rows x 253 columns]
    二人以上の世帯_支出金額[円] Unnamed: 1 Unnamed: 2  ...     245     246     247
183             181          -          -  ...     696     831     717
184             182          -          -  ...    9612    8368    8528
185             183          -          -  ...  252017  273699  266897
186        

### 日付の作成

In [ ]:
# データに存在する西暦のリストの作成
year_data = df.iloc[0].dropna() #欠損値を持つ行または列を削除する、デフォルトは行削除、引数(axis)に1を渡すと列を削除する
year_data_list = [int(year.replace('年', '')) for year in list(year_data)]
# データに存在する最終月の数値の作成
end_month = df.iloc[2, -1]
end_month = int(end_month.replace('月', ''))
print(year_data_list)
print(end_month)
# データに含まれる月数をカウントし、月数をdate_range関数に渡して日付を作成する
month_count = (len(year_data_list) - 1) * 12 + end_month
date_list = pd.date_range(f"{year_data_list[0]}", periods=month_count, freq="M")

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
7


#### date_range関数

- 一定頻度での日付を作成する関数
- 第一引数：日付作成の開始日
- 第二引数：期間(今回は合計月数)
- 第三引数：頻度(今回は月)

### 必要となるデータの抽出

In [ ]:
kakei_data = df.iloc[3:, 5:]
print(kakei_data)

       Unnamed: 5       1       2       3  ...     244     245     246     247
3    世帯数分布(抽出率調整)   10000   10000   10000  ...   10000   10000   10000   10000
4           集計世帯数    7887    7942    7934  ...    7534    7430    7489    7521
5         世帯人員(人)    3.32    3.32    3.32  ...    2.96    2.96    2.95    2.95
6      18歳未満人員(人)    0.74    0.75    0.75  ...    0.58    0.58    0.57    0.57
7      65歳以上人員(人)    0.52    0.53    0.53  ...    0.86    0.85    0.86    0.86
..            ...     ...     ...     ...  ...     ...     ...     ...     ...
183      会費・つきあい費    1561    1441    1495  ...     751     696     831     717
184      その他の教養娯楽    7063    6370    8068  ...    8603    9612    8368    8528
185      消費支出(再掲)  309621  290663  335341  ...  267922  252017  273699  266897
186         基礎的支出  184372  170045  186202  ...  159426  156494  165638  160605
187         選択的支出  125248  120618  149139  ...  108497   95523  108061  106293

[185 rows x 248 columns]


### 項目の追加と転置

In [ ]:
kakei_data = kakei_data.set_index('Unnamed: 5') # インデックスに設定
kakei_data.columns = date_list # 作成した日付をカラム設定
kakei_data.index.name = '' # インデックス名を消去
kakei_data = kakei_data.T # 行列転置を行うtransposeメソッドのシュガーシンタックス
kakei_data['date'] = kakei_data.index # date列の追加
kakei_data = kakei_data.reset_index(drop=True) # dropにTrueを渡すことで前にあったインデックスのデータをデータフレームに加えない様にする
kakei_data.to_csv('kakei_data.csv')
print(kakei_data)

    世帯数分布(抽出率調整) 集計世帯数 世帯人員(人) 18歳未満人員(人)  ... 消費支出(再掲)   基礎的支出   選択的支出       date
0          10000  7887    3.32       0.74  ...   309621  184372  125248 2000-01-31
1          10000  7942    3.32       0.75  ...   290663  170045  120618 2000-02-29
2          10000  7934    3.32       0.75  ...   335341  186202  149139 2000-03-31
3          10000  7922    3.32       0.75  ...   335276  173441  161835 2000-04-30
4          10000  7928    3.31       0.75  ...   308566  177153  131413 2000-05-31
..           ...   ...     ...        ...  ...      ...     ...     ...        ...
242        10000  7489    2.96       0.57  ...   292214  170401  121813 2020-03-31
243        10000  7534    2.96       0.58  ...   267922  159426  108497 2020-04-30
244        10000  7430    2.96       0.58  ...   252017  156494   95523 2020-05-31
245        10000  7489    2.95       0.57  ...   273699  165638  108061 2020-06-30
246        10000  7521    2.95       0.57  ...   266897  160605  106293 2020-07-31

[24

## 日本の新型コロナウイルス感染者数データの前処理

### Pandasで内容を確認し、データ加工の方針を決定

In [ ]:
pd.set_option('max_columns', 60) # デフォルト20なので、60に拡張
csv_url = 'https://dl.dropboxusercontent.com/s/6mztoeb6xf78g5w/COVID-19.csv'
df = pd.read_csv(csv_url, low_memory=False) # low_memory=Falseでpandasが全データを加味した型推測をしなくなる
print(df.shape)
print(df.info())
print(df.head())
print(df.tail())

(96792, 54)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96792 entries, 0 to 96791
Data columns (total 54 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   通し                96792 non-null  int64  
 1   厚労省NO             3694 non-null   object 
 2   無症状病原体保有者         921 non-null    object 
 3   国内                302 non-null    object 
 4   チャーター便            11 non-null     object 
 5   年代                96792 non-null  object 
 6   性別                96792 non-null  object 
 7   確定日               96792 non-null  object 
 8   発症日               43442 non-null  object 
 9   受診都道府県            96792 non-null  object 
 10  居住都道府県            96787 non-null  object 
 11  居住管内              635 non-null    object 
 12  居住市区町村            58650 non-null  object 
 13  キー                96789 non-null  object 
 14  発表                96298 non-null  object 
 15  都道府県内症例番号         92908 non-null  object 
 16  市町村内症例番号          15443 non-

### 扱いたいデータを抽出

In [ ]:
selected_cols = ['年代', '性別', '確定日', '発症日', '居住都道府県', 'X', 'Y']
df1 = df[selected_cols]
df1 = df1.dropna(how='all') # 行の全てがNaNである場合は行を消去
df1['count'] = 1 # count列を作成し、そこには1を設定
for col in df1.columns:
  print(f'{col}:{df1[col].unique()}')

年代:['30 ' '40 ' '60 ' '50 ' '20 ' '80 ' '70 ' '10 ' '0-10' '不明' '90 ' '50'
 '90' '70' '30' '20' '40' '80' '60' '10' '-' '80以上']
性別:['男性' '女性' '不明' '女児' '男児' '男性\u3000' '⼥性' '女性\xa0' '不明性']
確定日:['1/15/2020' '1/24/2020' '1/25/2020' '1/26/2020' '1/28/2020' '1/29/2020'
 '1/30/2020' '1/31/2020' '2/1/2020' '2/4/2020' '2/5/2020' '2/7/2020'
 '2/10/2020' '2/11/2020' '2/13/2020' '2/14/2020' '2/15/2020' '2/16/2020'
 '2/17/2020' '2/18/2020' '2/19/2020' '2/20/2020' '2/21/2020' '2/22/2020'
 '2/23/2020' '2/24/2020' '2/25/2020' '2/26/2020' '2/27/2020' '2/28/2020'
 '2/29/2020' '3/1/2020' '3/2/2020' '3/3/2020' '3/4/2020' '3/5/2020'
 '3/6/2020' '3/7/2020' '3/8/2020' '3/9/2020' '3/10/2020' '3/11/2020'
 '3/12/2020' '3/13/2020' '3/14/2020' '3/15/2020' '3/16/2020' '3/17/2020'
 '3/18/2020' '3/19/2020' '3/20/2020' '3/21/2020' '3/22/2020' '3/23/2020'
 '3/24/2020' '3/25/2020' '3/26/2020' '3/27/2020' '3/28/2020' '3/29/2020'
 '3/30/2020' '3/31/2020' '4/1/2020' '4/2/2020' '4/3/2020' '4/4/2020'
 '4/5/2020' '4/6/2020

### 年代の修正

In [ ]:
df1['年代'] = df1['年代'].apply(lambda x: x.replace('0-10', '1'))
df1['年代'] = df1['年代'].apply(lambda x: x.replace('不明', '99'))

# 当本の出版日以降に新たに追加されたフォーマット
df1['年代'] = df1['年代'].apply(lambda x: x.replace('-', '99'))
df1['年代'] = df1['年代'].apply(lambda x: x.replace('80以上', '99'))

# 確認用(上記の様にフォーマットが異なる値が追加されるため)
# print(df1[df1['年代'].str.contains('.*以上.*')])

df1['年代'] = df1['年代'].apply(lambda x: int(x))

# どんなデータが存在するかの確認(print要らず)
df1['年代'].unique()

# データの保存
df1.to_csv('covid_19_update.csv')

## 日本の財政データの前処理

In [ ]:
# Pandasで内容を確認し、データ加工の方針を決定
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)
excel_url = 'https://www.mof.go.jp/budget/reference/statistics/24.xls'
df=pd.read_excel(excel_url)
print(df.shape)
print(df.info())
print(df.head())
print(df.tail())
print(df)

WARNING *** file size (44668) not 512 + multiple of sector size (512)
(57, 26)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   17 non-null     object 
 1   Unnamed: 1   37 non-null     object 
 2   Unnamed: 2   37 non-null     object 
 3   Unnamed: 3   0 non-null      float64
 4   Unnamed: 4   53 non-null     object 
 5   Unnamed: 5   53 non-null     object 
 6   Unnamed: 6   53 non-null     object 
 7   Unnamed: 7   53 non-null     object 
 8   Unnamed: 8   53 non-null     object 
 9   Unnamed: 9   53 non-null     object 
 10  Unnamed: 10  53 non-null     object 
 11  Unnamed: 11  53 non-null     object 
 12  Unnamed: 12  53 non-null     object 
 13  Unnamed: 13  53 non-null     object 
 14  Unnamed: 14  53 non-null     object 
 15  Unnamed: 15  53 non-null     object 
 16  Unnamed: 16  53 non-null     object 
 17  Unnamed: 17  53

In [ ]:
# 必要な無い行、列を削除
df1 = df.iloc[5:56, :]
df1 = df1.drop(['Unnamed: 1', 'Unnamed: 3'], axis=1) # 不要な列を削除(axis=0が行、axis=1が列)
df1 = df1.reset_index(drop=True)
df1.columns = [f'col_{i}' for i in range(len(df1.columns))] # インデックスの値の接頭辞(col_)を付加

In [ ]:
# 横棒が入っているデータを0に置換
replace_strs = df1.iloc[46, 4:].unique()
df1 = df1.replace(replace_strs, 0)

## 歳出項目を大項目と詳細で管理

In [ ]:
# 各行に大項目を持つ列を作成
df1['main_title'] = df1['col_0'].fillna(method='ffill') # 欠損値(NaN)は前方の値に置換

In [ ]:
# 大項目のみのデータを作成
col0_has_value = df1['col_0'].dropna()
df_big = df1.iloc[col0_has_value.keys(), :].dropna(1) # dropna(1)は、dropna(axis=1)のシュガーシンタックス
col_data = [year for year in range(1997, 2019)] # データは平成9年(1997)～平成30年(2018)まで)までだが、range関数は2019指定して2018までを生成する
col_data.insert(0, 'title')
col_data.insert(len(col_data), 'main_title')
df_big.columns = col_data
df_big = df_big.reset_index(drop=True)

In [ ]:
# 詳細データを作成
col1_has_value = df1['col_1'].dropna()
df_detail = df1.iloc[col1_has_value.keys(), :].dropna(1)
df_detail.columns = col_data
df_add = df_big.iloc[[2, 8, 9, 10, 11, 12, 13], :]
df_detail = pd.concat([df_detail, df_add])
df_detail = df_detail.reset_index(drop=True)
df_detail.to_csv('zaisei_detail.csv')

# WEB+DB PRESS vol.118 COVID-19 4. 整形したデータの可視化

- Plotly ExpressはPlotlyのラッパーライブラリ
- 少ないコードで様々なグラフが利用可能なのが特徴

In [ ]:
import plotly.express as px
fig = px.line(x=[1,2,3,4,5], y=[3,5,2,4,6], title='plotly-expressのグラフ')
fig.show()

In [ ]:
import pandas as pd
df = pd.read_csv('kakei_data.csv', index_col=0, parse_dates=['date'])
fig = px.line(df, x='date', y='スポーツ', title='スポーツ支出')
fig.show()

In [ ]:
df['year'] = df['date'].map(lambda x: x.year)
df['month'] = df['date'].map(lambda x:x.month)
fig = px.line(df, x='month', y='スポーツ', color='year', title='各年のスポーツ支出')
fig.show()

In [ ]:
# データを計算して棒グラフで描画
df_yoy = df.copy() #deepコピー(b=aは参照渡し, b=a.iloc[:]はデータ生成(=deepコピー))
df_yoy.index = df_yoy['date']
df_yoy = df_yoy.loc[:, ~df_yoy.columns.isin(['date'])] # ~(チルダ)はnot演算子と等価
df_yoy = df_yoy.pct_change(12) # 12行前(1年前)

df_yoy2004 = df_yoy.loc['2020-4'].T.sort_values(by='20200430')
df_yoy2004.columns = ['前年比']
fig = px.bar(df_yoy2004, x=df_yoy2004.index, y='前年比', title='2020年4月の前年比', width=1000)
fig.show()

#df_yoy2005 = df_yoy.loc['2020-5'].T.sort_values(by='20200531')
#df_yoy2005.columns = ['前年比']
#fig = px.bar(df_yoy2005, x=df_yoy2005.index, y='前年比', title='2020年5月の前年比', width=1000)
#fig.show()

#df_yoy2006 = df_yoy.loc['2020-6'].T.sort_values(by='20200630')
#df_yoy2006.columns = ['前年比']
#fig = px.bar(df_yoy2006, x=df_yoy2006.index, y='前年比', title='2020年6月の前年比', width=1000)
#fig.show()

#df_yoy2007 = df_yoy.loc['2020-7'].T.sort_values(by='20200731')
#df_yoy2007.columns = ['前年比']
#fig = px.bar(df_yoy2007, x=df_yoy2007.index, y='前年比', title='2020年7月の前年比', width=1000)
#fig.show()

## groupbyメソッドを用いて集計したデータを可視化：感染者データ

In [ ]:
# 年代別感染者数を円グラフで描画
covid = pd.read_csv('covid_19_update.csv', index_col=0, parse_dates=['確定日', '発症日'])
covid_age = covid.groupby('年代', as_index=False).count()
px.pie(covid_age, names='年代', values='count')

# データを詳しく見たい時用のスニペット
#pd.set_option('display.max_rows', 100000)
#pd.set_option('display.max_columns', 100000)
#covid2 = covid.dropna(subset=['居住都道府県'])
#covid2 = covid2[covid2['居住都道府県'].str.contains('中華人民共和国')]
#covid2.head(2).append(covid2.tail(2))

In [ ]:
# 複数の要素をサンバーストグラフ(円グラフ)で描画
covid_data = covid.groupby(['居住都道府県', '年代', '性別'], as_index=False).count()
fig = px.sunburst(covid_data, path=['居住都道府県', '年代', '性別', 'count'], values='count')
fig.show()

## データを整然データに更新して可視化：財政データ

### 整然データとは

- "1行"を見るとデータの意味合いが理解できるデータ
- ロングフォームデータと呼ばれることもある
- 人間にとって分かりやすいデータが整然データとは限らない(むしろそうでないことが多い)
- データベースで考えると、非正規化している感じになる場合もある
- See. [整然データとは何か](https://id.fnshr.info/2017/01/09/tidy-data-intro/)
  - [上記リンク先より抜粋]正規化は、整然化し、矛盾を排除するのに有用である。しかし、リレーショナルデータを直接扱うデータ解析ツールはほとんどないため、通常、分析では、非正規化をしたりデータセットを1つの表に統合したりすることも必要とされる。

### melt関数

- 主成分となる値を固定値とみなし、それ以外を変数要素と捉えレコードに分解する関数
- See. [pandas のmelt関数の解説](https://masamunetogetoge.com/pandas-melt

In [ ]:
# 雑然(messy)データを整前(tidy)データに更新
zaisei = pd.read_csv('zaisei_detail.csv', index_col=0)
zaisei_melt = zaisei.melt(id_vars=['title', 'main_title']) #変数を合算しない場合は引数は固定化したい値だけでOK

In [ ]:
# 整然データをツリーマップで描画
zaisei_melt['board'] = '日本の財政データ'
px.treemap(zaisei_melt, path=['board', 'variable', 'main_title', 'title'], values='value')

In [ ]:
# main_titleとvariableを入れ替えるだけで、見え方が全く変わる
px.treemap(zaisei_melt, path=['board', 'main_title', 'variable', 'title'], values='value')

# ブラウザで動くダッシュボードの作成

- コンポーネントと呼ばれるツールを組み合わせてDashアプリケーションを作成
- これらをうまく組み合わせる事で、インタラクティブなアプリケーションを作成可能

## コンポーネントを構成するパッケージ

|コンポーネントの種類|概要|
|--------------------|----|
|Dash HTML Components|HTML要素を作成する|
|Dash Core Components|スライダ、ドロップダウン、グラフなどを作成する|
|Dash Data Table|インタラクティブなテーブルを作成する|
|Dash Bio|バイオインフォマティクス向けグラフを作成する|
|Dash DAQ|IoT端末などからデータ収集用ツールを提供する|
|Dash Canvas|画像を処理する|
|Dash Cytoscape|ネットワークを可視化する|

In [ ]:
# JupyterDashをGoogle Colaboratoryで使える様にする
!pip install jupyter_dash

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 81kB 3.7MB/s 
     |████████████████████████████████| 1.0MB 25.7MB/s 
     |████████████████████████████████| 3.5MB 31.7MB/s 
     |████████████████████████████████| 194kB 31.0MB/s 
     |████████████████████████████████| 1.8MB 34.7MB/s 
     |████████████████████████████████| 358kB 37.8MB/s 
  Created wheel for ansi2html: filename=ansi2html-1.5.2-cp36-none-any.whl size=13862 sha256=753a0edf383a2c806f8ffa82c9da139d94e7652031b59ae489ce196de2d83a4d
  Stored in directory: /root/.cache/pip/wheels/f9/ea/e0/1ed3c3945ce073f930f3e5aa2b9bbadb03cd2efc8f8695cd61
  Created wheel for dash: filename=dash-1.17.0-cp36-none-any.whl size=83728 sha256=f220b4dea39379b071463f2d1e769d2dc42d1d7f6545e6074c47403b34ad2852
  Stored in directory: /root/.cache/pip/wheels/e0/32/13/8a569b638a79cf80a4a721f56040d91198adec28e3bc8123fc
  Created wheel for flask-compress: filename=Flask_Compress-1.8.0-cp36-none-any.whl size=7215

## HTML要素を作成するパッケージ：Dash HTML Components

- Dash HTML ComponentsはすべてのHTMLタグをクラスとして提供するパッケージ
- HTMLタグの最初の文字を大文字にしたクラスを提供

In [ ]:
# Hello World アプリケーションの作例

from jupyter_dash import JupyterDash
import dash_html_components as html

app = JupyterDash(__name__) # JupyterDashインスタンスを作成
app.layout = html.H1('HELLO WEB+DB PRESS')
app.run_server(mode='inline') # flaskサーバを起動(通常は8050ポート起動)

In [ ]:
# 複数コンポーネントを組み合わせた作例

app = JupyterDash(__name__)
app.layout = html.Div([
  html.P('Show Image', style={'backgroundColor': 'red', 'color': 'white'}),
  html.Img(src='https://3.bp.blogspot.com/--PL6hlrPIPo/Uku_GiUXI6I/AAAAAAAAYuI/aOQxoDd6Oy8/s400/kankou_kinkakuji.png')
])
app.run_server(mode='inline')

## ツールを提供するパッケージ：Dash Core Components

- ドロップダウンやグラフなどを提供

In [ ]:
# Graph コンポーネント

import dash_core_components as dcc
import pandas as pd
import plotly.express as px

df = pd.read_csv('kakei_data.csv', index_col=0, parse_dates=['date'])
app = JupyterDash(__name__)
app.layout = html.Div([
  html.H1('Graphコンポーネントの利用'),
  dcc.Graph(
      figure=px.line(
          df, x='date', y='スポーツ', title='スポーツ支出'
      )
  )
])
app.run_server(mode='inline')

In [ ]:
# Markdown コンポーネント

app = JupyterDash(__name__)

app.layout = dcc.Markdown('''
  # Markdownを使います
  ## Dash Components
  ### dash html Components
  - htmlタグを提供
  ### dash core Components
  - 様々なツールを提供
''', style={
    'color': 'green', 'backgroundColor': 'black', 'padding': '2%'
})

app.run_server(mode='inline')

In [ ]:
# 選択 / 入力コンポーネント

from datetime import date

app = JupyterDash(__name__)
four_styles = {
    'margin': '3%', 
    'width': '40%', 
    'display': 'inline-block', # コンポーネントを横に並べたいため
    'verticalAlign': 'top' # コンポーネントの高さを合わせるため
}
selection = {'tokyo', 'osaka', 'fukuoka'}

app.layout = html.Div([
    html.Div([dcc.Dropdown(options=[{'value': i, 'label': i} for i in selection], value='tokyo')], style=four_styles),
    html.Div([dcc.Input(placeholder='文字を入力してください')], style=four_styles),
    html.Div([dcc.DatePickerRange(start_date=date(2017, 12, 18), end_date=date(2020, 6, 22),),], style=four_styles),
    html.Div([html.Button('HELLO PUSH ME')], style=four_styles)
])

app.run_server(mode='inline')

## Dashにインタラクティブ要素を与える「コールバック」

- インタラクティブ要素を与える機能

In [ ]:
# コールバックの基本的な作り方

from dash.dependencies import Input, Output, State
app = JupyterDash(__name__)

graph_type_list = [px.line, px.scatter, px.bar]

app.layout = html.Div([
    dcc.Dropdown(
        id = 'my_dropdown',
        options = [{'label': type_.__name__, 'value': num}
            for num, type_ in enumerate(graph_type_list)], 
            value = 0),
    html.Button(
        id='my_button',
        children='Update Graph' # この場合はボタン名になる
    ),
    dcc.Graph(
        id='my_graph',
    ),
])

# app.callbackデコレータを作成
# - Output/Input/Stateに指定するキー名は、描画するHTMLのidに紐づく必要有
# - Outputは複数であればリストを設定する必要あるが、InputとStateは単数でもリストを指定する必要有
@app.callback(Output('my_graph', 'figure'), # Output:コールバック関数の戻り値の出力先指定
              [Input('my_button', 'n_clicks')], # Input:呼び出しを指定
              [State('my_dropdown', 'value')]) # State:状態をコンポーネント保持するコンポーネント
def update_graph(n_clicks, selected_value): # 関数名と引数名はなんでもいい
  return graph_type_list[selected_value](x=[1,2,3,4,5], y=[1,2,3,4,5])

app.run_server(mode='inline')

<IPython.core.display.Javascript object>